In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.pyplot import figure
from matplotlib.dates import DateFormatter

import numpy as np
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import math

In [ ]:
tweets_sentiments = pd.read_csv('../data/interim/bitcoin_all_tweets_sentiments_merged_20210101-20210930.csv', index_col = 0)

In [3]:
#1107812/1107811
tweets_sentiments = pd.read_csv('../data/interim/bitcoin_all_tweets_sentiments_20210101-20210930.csv', index_col = 0)
tweets_sentiments_next = pd.read_csv('../data/interim/bitcoin_all_tweets_sentiments_20210201-20210531.csv', index_col = 0)
tweets_sentiments_next = tweets_sentiments_next[1107811:]
tweets_sentiments_full = pd.concat([tweets_sentiments, tweets_sentiments_next])

filename = '../data/interim/bitcoin_all_tweets_sentiments_merged_20210101-20210930.csv'
tweets_sentiments_full.to_csv(filename, mode='w')


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [ ]:
tweets_sentiments


In [ ]:
tweets_sentiments['dateTime']= pd.to_datetime(tweets_sentiments['dateTime'])


In [ ]:
tweets_sentiments = tweets_sentiments[['id', 'dateTime', 'prediction']]

tweets_sentiments.info()



In [ ]:
thirty_mins_span = tweets_sentiments.groupby('prediction').resample('30min', on='dateTime').count()

one_h_span = tweets_sentiments.groupby('prediction').resample('1H', on='dateTime').count()

two_h_span = tweets_sentiments.groupby('prediction').resample('2H', on='dateTime').count()

daily_span = tweets_sentiments.groupby('prediction').resample('D', on='dateTime').count()

fifteen_mins_span = tweets_sentiments.groupby('prediction').resample('15min', on='dateTime').count()

In [ ]:
thirty_mins_span = thirty_mins_span.unstack('prediction', fill_value=0)
thirty_mins_span = thirty_mins_span[('id')]

one_h_span = one_h_span.unstack('prediction', fill_value=0)
one_h_span = one_h_span[('id')]

two_h_span = two_h_span.unstack('prediction', fill_value=0)
two_h_span = two_h_span[('id')]

daily_span = daily_span.unstack('prediction', fill_value=0)
daily_span = daily_span[('id')]

fifteen_mins_span = fifteen_mins_span.unstack('prediction', fill_value=0)
fifteen_mins_span = fifteen_mins_span[('id')]


In [ ]:
two_h_span.index.name = None
two_h_span.columns.name = 'index'

#two_h_span.loc['2021-09-10':'2021-09-30']
daily_span.index.name = None
daily_span.columns.name = 'index'

#daily_span.loc['2021-09-10':'2021-09-30']

one_h_span.index.name = None
one_h_span.columns.name = 'index'

thirty_mins_span.index.name = None
thirty_mins_span.columns.name = 'index'


fifteen_mins_span.index.name = None
fifteen_mins_span.columns.name = 'index'



In [ ]:
def calc_percentage(df):
    df['total'] = df['negative'] + df['neutral'] + df['positive']
    df['pos_percent'] = df['positive'] / df['total'] * 100 
    df['neu_percent'] = df['neutral'] / df['total'] * 100
    df['neg_percent'] = df['negative'] / df['total'] * 100

calc_percentage(thirty_mins_span)
calc_percentage(one_h_span)
calc_percentage(two_h_span)
calc_percentage(daily_span)
calc_percentage(fifteen_mins_span)


In [ ]:
daily_span

In [ ]:
plot = daily_span

# Define the upper limit, lower limit, interval of Y axis and colors
y_LL = 0
y_UL = int(plot.iloc[:, 1:].max().max()*1.1)
#y_UL = 50000
y_interval = 4000
mycolors = ['tab:red', 'tab:blue', 'tab:green', 'tab:orange']    

# Draw Plot and Annotate
fig, ax = plt.subplots(1,1,figsize=(16, 9), dpi= 80)    

columns = plot.columns[:3]  
for i, column in enumerate(columns):    
    valt = plot[column].values
    plt.plot(plot.index.values, plot[column].values, lw=1.5, color=mycolors[i])    
    plt.text(plot.shape[0]+1, plot[column].values[-1], column, fontsize=14, color=mycolors[i])

# Draw Tick lines  
for y in range(y_LL, y_UL, y_interval):    
    plt.hlines(y, xmin=0, xmax=71, colors='black', alpha=0.3, linestyles="--", lw=0.5)

# Decorations    
plt.tick_params(axis="both", which="both", bottom=False, top=False,    
                labelbottom=True, left=False, right=False, labelleft=True)        

# Lighten borders
plt.gca().spines["top"].set_alpha(.3)
plt.gca().spines["bottom"].set_alpha(.3)
plt.gca().spines["right"].set_alpha(.3)
plt.gca().spines["left"].set_alpha(.3)

plt.title('a sample title', fontsize=22)
plt.yticks(range(y_LL, y_UL, y_interval), [str(y) for y in range(y_LL, y_UL, y_interval)], fontsize=12)    
plt.xticks(range(0, plot.shape[0], 24), plot.index.values[::24], horizontalalignment='left', fontsize=12)    


plt.ylim(y_LL, y_UL)    
plt.xlim(-2, 80)    
plt.show()

In [ ]:
plot = daily_span

x = plot.datetime.values

pos = plot['pos_percent']
neu = plot['neu_percent']
neg = plot['neg_percent']

mycolors = ['tab:red', 'tab:blue', 'tab:green']    
figure(figsize = (15, 4), dpi = 80)
plt.title('Tweets\' Daily Sentiments (Percentage)', loc='center')
plt.plot(x,pos, color=mycolors[2], label="positive")
plt.plot(x,neu, color=mycolors[1], label="neutral")
plt.plot(x,neg, color=mycolors[0], label="negative")
plt.legend(loc="upper right")
plt.show()

In [ ]:
thirty_mins_span = thirty_mins_span.reset_index().rename(columns={'index':'datetime'})
one_h_span = one_h_span.reset_index().rename(columns={'index':'datetime'})
two_h_span = two_h_span.reset_index().rename(columns={'index':'datetime'})
daily_span = daily_span.reset_index().rename(columns={'index':'datetime'})
fifteen_mins_span = fifteen_mins_span.reset_index().rename(columns={'index':'datetime'})


In [ ]:
bitcoin_prices = pd.read_csv('../data/external/Bitstamp_BTCUSD_2021_minute_final.csv', header = 1)


In [ ]:
bpd = bitcoin_prices[['date', 'open', 'Volume BTC']]
bpd['date'] = pd.to_datetime(bpd["date"], format="%Y-%m-%d %H:%M:%S", errors='coerce', utc=True)
#bpd['Date'] = bpd['SPdateTime'].dt.strftime('%Y-%m-%d')
bpd = bpd.set_index('date')
#bpd.reset_index()

bpd.info()
bpd.head()


In [ ]:
import datetime

def fetch_prices(df, span_mins):
    c_dts = df.datetime
    dts_1 = c_dts + datetime.timedelta(minutes=span_mins)
    dts_2 = c_dts + datetime.timedelta(minutes=(span_mins*2))
    dts_3 = c_dts + datetime.timedelta(minutes=(span_mins*3))
    dts_4 = c_dts + datetime.timedelta(minutes=(span_mins*4))

    df['cp'] = bpd.loc[c_dts].reset_index()['open']
    df['1spanp'] = bpd.loc[dts_1].reset_index()['open']
    df['2spanp'] = bpd.loc[dts_2].reset_index()['open']
    df['3spanp'] = bpd.loc[dts_3].reset_index()['open']
    df['4spanp'] = bpd.loc[dts_4].reset_index()['open']

    df['c_p_change'] = (df['1spanp'] - df['cp']) / df['cp']
    df['shift1_p_change'] = (df['2spanp'] - df['1spanp']) / df['1spanp']
    df['c_plus_shift1_p_change'] = (df['2spanp'] - df['cp']) / df['cp']

fetch_prices(thirty_mins_span, 30)
fetch_prices(one_h_span, 60)
fetch_prices(two_h_span, 120)
fetch_prices(fifteen_mins_span, 15)




In [ ]:
def fetch_trade_vol(df, span_mins):
    c_dts = df.datetime
    vols = bpd.resample(str(span_mins) + 'min').sum()['Volume BTC']
    df['vol_btc'] = vols.loc[c_dts].reset_index()['Volume BTC']
    

fetch_trade_vol(thirty_mins_span, 30)
fetch_trade_vol(one_h_span, 60)
fetch_trade_vol(two_h_span, 120)
fetch_trade_vol(fifteen_mins_span, 15)



In [ ]:
def get_df_with_btc_vol_change (df):    
    #vol_diff = df[['vol_btc']].diff(periods = 2)
    vol_perc_diff = df[['vol_btc']].pct_change(periods = 2)
    vol_perc_diff.replace([np.inf, -np.inf], np.nan, inplace=True)
    vol_perc_difs = vol_perc_diff.shift(-1)
    vol_perc_difs.rename(columns={'vol_btc': 'vol_btc_change'}, inplace=True)
    return df.join(vol_perc_difs)

thirty_mins_span = get_df_with_btc_vol_change(thirty_mins_span)
one_h_span = get_df_with_btc_vol_change(one_h_span)
two_h_span = get_df_with_btc_vol_change(two_h_span)
fifteen_mins_span = get_df_with_btc_vol_change(fifteen_mins_span)


In [ ]:
def get_df_with_post_sentiment_change (df):    
    sentiment_diff = df[['pos_percent', 'neu_percent', 'neg_percent']].diff(periods = 1)
    sent_difs = sentiment_diff.shift(-1)
    sent_difs.rename(columns={'pos_percent': 'pos_post_change', 'neu_percent': 'neu_post_change', 'neg_percent':'neg_post_change'}, inplace=True)
    return df.join(sent_difs)

thirty_mins_span = get_df_with_post_sentiment_change(thirty_mins_span)
one_h_span = get_df_with_post_sentiment_change(one_h_span)
two_h_span = get_df_with_post_sentiment_change(two_h_span)
fifteen_mins_span = get_df_with_post_sentiment_change(fifteen_mins_span)

In [ ]:

thirty_diff = thirty_mins_span[['pos_percent', 'neu_percent', 'neg_percent']].diff(periods = 2)
thirty_difs = thirty_diff.shift(-1)
thirty_difs.rename(columns={'pos_percent': 'pos_change', 'neu_percent': 'neu_change', 'neg_percent': 'neg_change'}, inplace=True)
thirty_mins_span_j = thirty_mins_span.join(thirty_difs)


one_h_diff = one_h_span[['pos_percent', 'neu_percent', 'neg_percent']].diff(periods = 2)
one_h_difs = one_h_diff.shift(-1)
one_h_difs.rename(columns={'pos_percent': 'pos_change', 'neu_percent': 'neu_change', 'neg_percent': 'neg_change'}, inplace=True)
one_h_span_j = one_h_span.join(one_h_difs)

two_h_diff = two_h_span[['pos_percent', 'neu_percent', 'neg_percent']].diff(periods = 2)
two_h_difs = two_h_diff.shift(-1)
two_h_difs.rename(columns={'pos_percent': 'pos_change', 'neu_percent': 'neu_change', 'neg_percent': 'neg_change'}, inplace=True)
two_h_span_j = two_h_span.join(two_h_difs)

fifteen_mins_diff = fifteen_mins_span[['pos_percent', 'neu_percent', 'neg_percent']].diff(periods = 2)
fifteen_mins_difs = fifteen_mins_diff.shift(-1)
fifteen_mins_difs.rename(columns={'pos_percent': 'pos_change', 'neu_percent': 'neu_change', 'neg_percent': 'neg_change'}, inplace=True)
fifteen_mins_span_j = fifteen_mins_span.join(fifteen_mins_difs)


In [ ]:
thirty_pdiff = thirty_mins_span[['pos_percent', 'neu_percent', 'neg_percent']].diff(periods = 1)
thirty_pdiff.rename(columns={'pos_percent': 'pos_pre_change', 'neu_percent': 'neu_pre_change', 'neg_percent': 'neg_pre_change'}, inplace=True)
thirty_mins_span_j = thirty_mins_span_j.join(thirty_pdiff)

one_h_pdiff = one_h_span[['pos_percent', 'neu_percent', 'neg_percent']].diff(periods = 1)
one_h_pdiff.rename(columns={'pos_percent': 'pos_pre_change', 'neu_percent': 'neu_pre_change', 'neg_percent': 'neg_pre_change'}, inplace=True)
one_h_span_j = one_h_span_j.join(one_h_pdiff)

two_h_pdiff = two_h_span[['pos_percent', 'neu_percent', 'neg_percent']].diff(periods = 1)
two_h_pdiff.rename(columns={'pos_percent': 'pos_pre_change', 'neu_percent': 'neu_pre_change', 'neg_percent': 'neg_pre_change'}, inplace=True)
two_h_span_j = two_h_span_j.join(two_h_pdiff)

fifteen_mins_pdiff = fifteen_mins_span[['pos_percent', 'neu_percent', 'neg_percent']].diff(periods = 1)
fifteen_mins_pdiff.rename(columns={'pos_percent': 'pos_pre_change', 'neu_percent': 'neu_pre_change', 'neg_percent': 'neg_pre_change'}, inplace=True)
fifteen_mins_span_j = fifteen_mins_span_j.join(fifteen_mins_pdiff)


In [ ]:
thirty_mins_span_j[['pos_percent', 'pos_pre_change', 'pos_change', 'pos_post_change', 'vol_btc_change' ]]


In [ ]:
def assign_shift_target(df):
    df['curr_up'] = np.where(df['1spanp'] >  df['cp'], 1, 0)
    df['shift1_up'] = np.where(df['2spanp'] >  df['1spanp'], 1, 0)
    
    df['shift2_up'] = np.where(df['3spanp'] >  df['2spanp'], 1, 0)
    df['shift3_up'] = np.where(df['4spanp'] >  df['3spanp'], 1, 0)
    df['2_up'] = np.where(df['2spanp'] >  df['cp'], 1, 0)
    df['3_up'] = np.where(df['3spanp'] >  df['cp'], 1, 0)
    df['4_up'] = np.where(df['4spanp'] > df['cp'], 1, 0)
    df['future2_up'] = np.where(df['3spanp'] >  df['1spanp'], 1, 0)
    df['future3_up'] = np.where(df['4spanp'] >  df['1spanp'], 1, 0)

assign_shift_target(thirty_mins_span_j)
assign_shift_target(one_h_span_j)
assign_shift_target(two_h_span_j)
assign_shift_target(fifteen_mins_span_j)

In [ ]:
one_h_span_j[['pos_percent', 'neu_percent', 'neg_percent']]

In [ ]:
import pytz
def filter_time(df):
    #pacific = pytz.timezone('US/Pacific')
    #pacific_dt = df.set_index('datetime').tz_convert(pacific)
    return df#[(pacific_dt.index.hour < 0) | (pacific_dt.index.hour > 3)]

def filter_columns(df):
    return df[['datetime', 'total', 'pos_percent', 'neu_percent', 'neg_percent', 'vol_btc', 'vol_btc_change', 'pos_change', 'neu_change', 'neg_change', 
    'pos_pre_change', 'neu_pre_change', 'neg_pre_change', 'curr_up', 
    'pos_post_change', 'neu_post_change', 'neg_post_change',
    'c_plus_shift1_p_change', 'shift1_p_change', 'c_p_change',
    'shift1_up', 'shift2_up', 'shift3_up', "2_up", "3_up", "4_up", "future2_up", "future3_up"]]

thirty_mins_span_final = filter_time(filter_columns(thirty_mins_span_j))
one_h_span_final = filter_time(filter_columns(one_h_span_j))
two_h_span_final = filter_time(filter_columns(two_h_span_j))
fifteen_mins_span_final = filter_time(filter_columns(fifteen_mins_span_j))




In [ ]:
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.dummy import DummyClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import (
    TimeSeriesSplit,
    cross_val_score,
    cross_validate,
    train_test_split,
)
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler


In [ ]:
#thirty_mins_span_final.shape
#one_h_span_final.shape
#two_h_span.shape


In [ ]:

thirty_train_df = thirty_mins_span_final[:math.floor(thirty_mins_span_final.shape[0] * 0.65)]
thirty_test_df = thirty_mins_span_final[math.floor(thirty_mins_span_final.shape[0] * 0.65):]

one_h_train_df = one_h_span_final[:math.floor(one_h_span_final.shape[0] * 0.65)]
one_h_test_df = one_h_span_final[math.floor(one_h_span_final.shape[0] * 0.65):]

two_h_train_df = two_h_span_final[:math.floor(two_h_span_final.shape[0] * 0.65)]
two_h_test_df = two_h_span_final[math.floor(two_h_span_final.shape[0] * 0.65):]

fifteen_mins_train_df = fifteen_mins_span_final[:math.floor(fifteen_mins_span_final.shape[0] * 0.65)]
fifteen_mins_test_df = fifteen_mins_span_final[math.floor(fifteen_mins_span_final.shape[0] * 0.65):]


In [ ]:
plot = daily_span

x = plot.datetime.values
pos_train = thirty_train_df['pos_percent']
neu_train = thirty_train_df['neu_percent']
neg_train = thirty_train_df['neg_percent']
#mycolors = ['tab:red', 'tab:blue', 'tab:green']    
figure(figsize = (15, 4), dpi = 80)
#plt.title('Tweets\' Daily Sentiments (Percentage)', loc='center')
#plt.plot(x,pos, color=mycolors[2], label="positive")
#plt.plot(x,neu, color=mycolors[1], label="neutral")
#plt.plot(x,neg, color=mycolors[0], label="negative")
pos_test = thirty_test_df['pos_percent']
neu_test = thirty_test_df['neu_percent']
neg_test = thirty_test_df['neg_percent']

plt.plot(pos_train, "r", label="train")
plt.plot(neu_train, "r")
plt.plot(neg_train, "r")
plt.plot(pos_test, "b", label="test")
plt.plot(neu_test, "b")
plt.plot(neg_test, "b")

plt.legend(loc="upper right")
plt.show()

In [ ]:
numeric_features = [
    "pos_pre_change",
    "neu_pre_change",
    "neg_pre_change",
    "pos_change",
    "neu_change",
    "neg_change",
    "neu_post_change",
    "pos_post_change",
    "neg_post_change",
    "pos_percent",
    "neu_percent",
    "neg_percent",
    "total",
    "vol_btc_change",
    "c_plus_shift1_p_change",
]
categorical_features = [
    
]
drop_features = [
    #targets
    "shift2_up",
    "shift3_up",
    "2_up",
    "3_up",
    "4_up",
    "future2_up",
    "future3_up",
    #"total",
    #unused features
    "curr_up",
    "shift1_up",
    "datetime",
    "vol_btc",
    #"vol_btc_change",
   "c_p_change",
   "shift1_p_change",
]

selected_train_df = one_h_train_df
selected_test_df = one_h_test_df
prediction_target = "shift2_up"


In [ ]:
def preprocess_features(
    train_df,
    test_df,
    numeric_features,
    categorical_features,
    drop_features,
    target
):

    all_features = set(numeric_features + categorical_features + drop_features)
    if set(train_df.columns) != all_features:
        print("Missing columns", set(train_df.columns) - all_features)
        print("Extra columns", all_features - set(train_df.columns))
        raise Exception("Columns do not match")

    numeric_transformer = make_pipeline(
        SimpleImputer(strategy="median"), StandardScaler()
    )
    '''categorical_transformer = make_pipeline(
        #SimpleImputer(strategy="constant", fill_value="?"),
        SimpleImputer(strategy="most_frequent"),
        OneHotEncoder(handle_unknown="ignore", sparse=False),
    )'''

    preprocessor = make_column_transformer(
        (numeric_transformer, numeric_features),
        #(categorical_transformer, categorical_features),
        ("drop", drop_features),
    )
    preprocessor.fit(train_df)
    '''ohe_feature_names = (
        preprocessor.named_transformers_["pipeline-2"]
        .named_steps["onehotencoder"]
        .get_feature_names()
        .tolist()
    )'''
    #new_columns = numeric_features + ohe_feature_names
    new_columns = numeric_features
    X_train_enc = pd.DataFrame(
        preprocessor.transform(train_df), index=train_df.index, columns=new_columns
    )
    X_test_enc = pd.DataFrame(
        preprocessor.transform(test_df), index=test_df.index, columns=new_columns
    )

    y_train = train_df[target]
    y_test = test_df[target]

    return X_train_enc, y_train, X_test_enc, y_test, preprocessor

In [ ]:
X_train_enc, y_train, X_test_enc, y_test, preprocessor = preprocess_features(
    selected_train_df,
    selected_test_df,
    numeric_features,
    categorical_features,
    drop_features,
    prediction_target
)

In [ ]:
X_train_enc.head()

In [ ]:
def score_lr_print_coeff(preprocessor, train_df, y_train, test_df, y_test, X_train_enc):
    lr_pipe = make_pipeline(preprocessor, LogisticRegression(max_iter=1000))
    lr_pipe.fit(train_df, y_train)
    print("Train score: {:.5f}".format(lr_pipe.score(train_df, y_train)))
    print("Test score: {:.5f}".format(lr_pipe.score(test_df, y_test)))
    lr_coef = pd.DataFrame(
        data=lr_pipe.named_steps["logisticregression"].coef_.flatten(),
        index=X_train_enc.columns,
        columns=["Coef"],
    )
    return lr_coef.sort_values(by="Coef", ascending=False)

In [ ]:
# 30mins, 0930-0720, train 0.55, test 0.53, 1 shift
score_lr_print_coeff(preprocessor, selected_train_df, y_train, selected_test_df, y_test, X_train_enc)

Precision is the fraction of cases that are fraudulent out of those that our classifier labelled as positive. The problem with a low precision is that our investigators will spend a lot of time investigating claims that are actually legitimate.

Recall is the fraction of fraudulent cases our classifier finds. The problem with a low recall is that we would be paying out on a lot of undetected fraudulent claims.

If we want a higher precision, so that if our classifier predicted going up, it is more likely to go up, but we detected less proportion of the cases that are going up. (less trading opportunity, higher accuracy)

If we want a higher recall, then among our prediction that the bitcoin is going up, its less likely to go up, but we detected more proportion of the cases that are going up. (more trading opportunity, less accuracy)

Therefore, we care more about precision? 

# 1h

Observations:
The recall of sementic models generally have higher recall compared to 
baseline priced models

<strong>Baseline</strong>
shift1_p_change	-0.039855
c_p_change	-0.144066

Train score: 0.53594
Test score: 0.52188 (0.521878	0.542045	0.491753	0.515676)

c_plus_shift1_p_change	-0.128164
Train score: 0.53019
Test score: 0.52668 (0.526681	0.548200	0.486598	0.515565)


Exp1
neu_change	0.166109
neg_pre_change	0.142978
neg_post_change	0.127850
neu_percent	0.030957
pos_pre_change	0.020187
neg_percent	0.020143
pos_post_change	0.018853
total	0.005838
pos_percent	-0.037233
pos_change	-0.064472
vol_btc_change	-0.083970
neu_post_change	-0.102580
neu_pre_change	-0.124964
neg_change	-0.152797

Train score: 0.51265
Test score: 0.53095 (0.530950	0.544828	0.570103	0.557179)

Exp2: exp1 + price factor "c_plus_shift1_p_change"
Train score: 0.52703
Test score: 0.52455 (0.5	0.524546	0.539778	0.552577	0.546103)

# 30mins
Observations: 
1. for 30 mins prices factors have a stronger influence on future price direction compared to 1h
2. Does it mean including prices as a factor for 30 mins' prediction will have a stronger impact? YES!
3. Including prices + sentiments factors increases all evaluation factors especially recall, compared to baseline
4. Including prices decrease the chance of overfitting the model

Baseline:
c_plus_shift1_p_change	-0.069482 (0.521751	0.528804	0.735113	0.615120)

c_p_change	-0.001419
shift1_p_change	-0.100080
Train score: 0.52206
Test score: 0.52389 (0.523886	0.532905	0.681725	0.598198)

Exp1
pos_change	0.168012
neg_pre_change	0.129002
neg_post_change	0.113397
neu_post_change	0.089122
vol_btc_change	0.079499
neu_pre_change	0.077909
neu_percent	0.030173
c_p_change	0.000646
total	-0.011629
pos_percent	-0.012835
neg_percent	-0.016781
shift1_p_change	-0.099523
neu_change	-0.099739
neg_change	-0.134134
pos_pre_change	-0.142063
pos_post_change	-0.155439
Train score: 0.52595
Test score: 0.52148 (0.521484	0.525637	0.815708	0.639308)

exp2
pos_change	0.173532
neg_pre_change	0.127890
neg_post_change	0.118353
neu_post_change	0.091813
neu_pre_change	0.080184
vol_btc_change	0.076809
neu_percent	0.030805
total	-0.010345
pos_percent	-0.012031
neg_percent	-0.018764
c_plus_shift1_p_change	-0.068613
neu_change	-0.102020
neg_change	-0.140124
pos_pre_change	-0.143314
pos_post_change	-0.161050

Train score: 0.51790
Test score: 0.52469 (0.5	0.524686	0.526021	0.866530	0.654644)

Exp3 (exp1/2 - price factors)
Train score: 0.51603
Test score: 0.51668 (0.516680	0.519724	0.926591	0.665929)


# 2hs

c_plus_shift1_p_change	-0.181843

Train score: 0.54917
Test score: 0.53042 (0.530416	0.537879	0.591667	0.563492)


neg_post_change	0.079504
neu_pre_change	0.069734
neg_percent	0.057452
pos_percent	0.008676
neg_change	0.005855
neu_change	0.004623
pos_change	0.004318
neg_pre_change	-0.005493
pos_post_change	-0.021715
neu_post_change	-0.024930
total	-0.036585
vol_btc_change	-0.039507
pos_pre_change	-0.059234
neu_percent	-0.061897

Train score: 0.52214
Test score: 0.50800 (0.5	0.508004	0.529052	0.360417	0.428748) (0.4 0.512273	0.512326	0.995833	0.676575)

       


In [ ]:
thresholds = np.arange(0.0, 1.0, 0.02)
lr_pipe1 = make_pipeline(preprocessor, LogisticRegression(max_iter=1000))
lr_pipe1.fit(selected_train_df, y_train)

pr_dict = {"threshold": [], "accuracy": [], "precision": [], "recall": [], "f1 score": []}
for threshold in thresholds:
    preds = lr_pipe1.predict_proba(selected_test_df)[:, 1] > threshold
    pr_dict["threshold"].append(threshold)
    pr_dict["accuracy"].append(accuracy_score(y_test, preds))
    pr_dict["precision"].append(precision_score(y_test, preds))
    pr_dict["recall"].append(recall_score(y_test, preds))
    pr_dict["f1 score"].append(f1_score(y_test, preds))

pd.DataFrame(pr_dict)

In [ ]:
from sklearn.metrics import plot_confusion_matrix
plot_confusion_matrix(
    lr_pipe1,
    selected_train_df,
    y_train,
    display_labels=["Down", "Up"],
    values_format="d",
    cmap=plt.cm.Blues,
)

In [ ]:
from sklearn.metrics import precision_recall_curve

precision, recall, thresholds = precision_recall_curve(
    y_test, lr_pipe1.predict_proba(selected_test_df)[:, 1]
)
plt.plot(precision, recall, label="logistic regression: PR curve")
plt.xlabel("Precision")
plt.ylabel("Recall")
plt.plot(
    precision_score(y_test, lr_pipe1.predict(selected_test_df)),
    recall_score(y_test, lr_pipe1.predict(selected_test_df)),
    "or",
    markersize=10,
    label="threshold 0.5",
)
plt.legend(loc="best");